## Reading the articles, token by token. 
(at least the characters match)

# Read in tokens (not necessary)
      # not necessary: token_list = ltf_to_sent.load_ltf(ltf_str)

In [28]:
from utils import ltf_to_sent
import tqdm
import csv
import os
import spacy
import tqdm
nlp = spacy.load("en_core_web_trf")

def get_raw_text_from_file(filepath):
  # /home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
    basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
    filepath = os.path.join(basepath, filepath)
    filepath += ".ltf.xml"
    text_file = open(filepath, "r", encoding="utf-8")
    ltf_str = text_file.read()
    text = ltf_to_sent.ltf_to_plaintext(ltf_str)
    return text
      
def get_gold_labels():
  # gold_labels_path = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/annotations/edl.tab'
  # gold_labels_path = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/utils/edl.tab'
  gold_labels_path = './edl.tab'
  lines = []
  with open(gold_labels_path, encoding="utf-8") as tsv:
    for line in csv.reader(tsv, dialect="excel-tab"): # dialect="excel-tab" or delimiter="\t" 
      lines.append(line)
  print("example lline:", lines[0])
  return lines


def main():
  labels = get_gold_labels()
  
  # init variables
  first_doc_name = labels[0][3].split(':')[0]
  last_doc_filename = first_doc_name
  count = 0 
  num_span_is_none = 0
  doc = nlp(get_raw_text_from_file(first_doc_name))
  doc.ents = []
  db = spacy.tokens.DocBin()

  
  # iterate thru all labels, saving docs as we go
  for _, num, span_text, doc_name, ent_gold, annot_list, NAM_type, const in tqdm.tqdm(labels):
    curr_doc_filename = doc_name.split(':')[0]
    if curr_doc_filename != last_doc_filename:
      # save the last spacy.doc, and load in the next spacy.doc. 
      last_doc_filename = curr_doc_filename
      db.add(doc)
      doc = nlp(get_raw_text_from_file(curr_doc_filename))
      doc.ents = []
      print("starting new doc")
      print("Doc: ", doc)
      # return
    
    # iterate thru the labels and set the spans.
    start_char, end_char = doc_name.split(':')[1].split('-')[0], doc_name.split(':')[1].split('-')[1]

    print("text:", span_text)
    print("ent_gold:", ent_gold)
    print(doc_name)
    print(curr_doc_filename)
    print(start_char, end_char)

    print("Doc ents:", doc.ents)
    
    span = doc.char_span(int(start_char), int(end_char)+1, label=ent_gold, alignment_mode="contract")
    print("span:",span)
    print("span type:", type(span))
    if span is None:
      num_span_is_none += 1
      continue
    if type(doc.ents) is not "NoneType":
      doc.ents = list(doc.ents) + [span]
    else:
      doc.ents = [span]
    
    # if count > 3:
    #   break
    # count += 1
  # end loop 
  db.to_disk("./train_data_with_ents.spacy")
  print("⚠️ num_span_is_none:",num_span_is_none)

main()



<>:71: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:71: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_2627883/1756204657.py:71: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if type(doc.ents) is not "NoneType":


example lline: ['NIST', 'annotation-0', 'Farmers', '20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6', 'U.S. farmers', 'PER;PER.Farmer', 'NOM', '1.0']


  2%|▏         | 71/4727 [00:00<00:06, 699.72it/s]

text: Farmers
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
0 6
Doc ents: ()
span: Farmers
span type: <class 'spacy.tokens.span.Span'>
text: U.S.
ent_gold: United States
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:118-121
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
118 121
Doc ents: (Farmers,)
span: U.S.
span type: <class 'spacy.tokens.span.Span'>
text: farmers
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:123-129
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
123 129
Doc ents: (Farmers, U.S.)
span: farmers
span type: <class 'spacy.tokens.span.Span'>
text: They
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:311-314
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
311 314
Doc ents: (Farmers, U.S., farmers)
span: They
span type: <class 'spacy.tokens.span.Span'>
text: Bloomberg
ent_gold: Bloomberg
20120211_WAPO_a6be1e92-50de-11e1-bd4

  3%|▎         | 141/4727 [00:00<00:28, 161.83it/s]

...


20130915_WAPO_b375f724-1e21-11e3-9ad0-96244100e647:3144-3149
20130915_WAPO_b375f724-1e21-11e3-9ad0-96244100e647
3144 3149
Doc ents: (Kerry, Israel, Syria, accord, Iran, JERUSALEM, U.S., government, Israel, deal, Syria, weapons, American, Iran, John F. Kerry, Israel, Benjamin Netanyahu, he, We, Iran, North Korea, others, Kerry, Netanyahu, Obama, ABC, Iran, its, us, weapons, Syria, Iran, Israel, our, He, he, Iranian, leadership, Obama, Iran, his, Syria, My, Iranians, they, we, Syria, Iran, Obama, Israel, deal, which, Syria, 1993 Chemical Weapons Convention, its, weapons, agreement, Damascus, suburb, U.S., officials, people, Israel, Syria, its, weapons, Tehran, Syria, ally, Israeli, diplomats, Syria, arms, arsenal, Israel, Netanyahu, Kerry, he, U.S., Syria, its, stockpile, Syrian, regime, its, weapons, Netanyahu, Kerry, their, our, region, Netanyahu, We, understandings, U.S., Russia, Syria, weapons, understandings, stocks, Syrian, regime, its, people, Netanyahu, Israel, Syria, Egypt, We,

 98%|█████████▊| 4648/4727 [00:38<00:00, 160.44it/s]

starting new doc
Doc:  Mexico hit again by strengthened Manuel Thursday, September 19, 2013 11:48:00 GMT-05:00 MEXICO CITY — Storm-battered Mexico absorbed more damage Thursday from the drenching Pacific system known as Manuel, which inflicted its second strike on the country in less than five days, this time at hurricane strength. Manuel and a system that has swamped Mexico’s gulf coast, Ingrid, are blamed for at least 97 deaths so far this week and have washed out roads, ruined crops and severed communications in wide swaths of the country. After delivering its first blow as a tropical storm Sunday in southwestern Mexico, Manuel returned to the Pacific and drifted northward, reorganizing and gathering new momentum. It made landfall again Thursday as a Category 1 hurricane and flooded fishing villages along the Sea of Cortez, in the northwestern state of Sinaloa. At least 100,000 people suffered damage to their homes, state authorities said. Mexico’s hardest-hit region has been the so

100%|██████████| 4727/4727 [00:38<00:00, 122.41it/s]

starting new doc
Doc:  Pakistan train derails in bombing, killing at least 7 Monday, October 21, 2013 03:49:00 GMT-05:00 ISLAMABAD, Pakistan — A passenger train derailed Monday in Pakistan’s restive Baluchistan province when a bomb detonated next to it, killing at least seven people, officials said. The Jaffar Express train was traveling from Rawalpindi to Quetta when the attack occurred mid-morning, about 300 miles from its destination. Riaz Ahmed, a railway police officer in Quetta, said the bomb had been planted on the tracks. But Pakistan’s railway minister, Khwaja Saad Rafiq, told local news media that authorities also are investigating whether a suicide bomber could be to blame. Seven passengers, including three brothers traveling together, were killed, and at least a dozen others were injured, Ahmed said. Four train cars flipped when the bomb detonated, he added. There was no assertion of responsibility for the bombing, but Pakistan’s military has been battling separatists for y

In [ ]:
""" DEPRICATED::  THIS IS THE BEST WAY TO READ THE LTF FILES. """

# list all files in the directory
basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
import os
for file in os.listdir("/home/rohit/Downloads/ltf_files/"):
  if file.endswith(".ltf.xml"):
    filepath = os.path.join(basepath, file)
    text_file = open(filepath,"r")
    ltf_str = text_file.read()
    
    ### Read in plain text
    text = ltf_to_sent.ltf_to_plaintext(ltf_str)
    start_token = 1009
    end_token = 1013
    print(text[start_token : end_token+1]) # NOTE the +1

In [15]:
"""
# Read in tokens (not necessary)
token_list = ltf_to_sent.load_ltf(ltf_str)
"""

'\n# Read in tokens (not necessary)\ntoken_list = ltf_to_sent.load_ltf(ltf_str)\n'

In [ ]:
""" Example code """

db = DocBin() # create a DocBin object

for text, annot in tqdm.tqdm(train): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)


In [ ]:
db.to_disk("./train.spacy") # save the docbin object

### STRETCH GOAL NOT REQUIRED: Parse mention.tab

We DON'T do localization. So just iterate thru the test set and classify each token in the `mention.tab` file into one of the `ontology_types.txt` classes. 

```
mention_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/mentions.tab"

ontology_types_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/utils/rufes/tools/rufes-evaluation/example/data/ontology_types.txt"
```

In [ ]:
import csv
# mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# We don'
mention_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/mentions.tab"
lines = []
with open(mention_path) as tsv:
  for line in csv.reader(tsv, dialect="excel-tab"): #You can also use delimiter="\t" rather than giving a dialect.
    lines.append(line)
print("example lline:", lines[0])

# return
# third field
# Field 7: mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# NIST	annotation-0	Farmers	20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6	U.S. farmers	PER;PER.Farmer	NOM	1.0
count = 0 
for _, num, text, doc_name, ent_gold, annot_list, NAM_type, const in tqdm.tqdm(lines):

# Parse edl.tab (gold training labels)

In [14]:

# Field 7: mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# NIST	annotation-0	Farmers	20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6	U.S. farmers	PER;PER.Farmer	NOM	1.0
  ''' add ents to doc '''
  # doc = nlp.make_doc(text)
  # ents = []
  # for start, end, label in annot["entities"]:
  #   span = doc.char_span(start, end, label=label, alignment_mode="contract")
  #   if span is None:
  #     print("Skipping entity")
  #   else:
  #     ents.append(span)
  # doc.ents = ents
  # db.add(doc)
  

example lline: ['NIST', 'annotation-0', 'Farmers', '20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6', 'U.S. farmers', 'PER;PER.Farmer', 'NOM', '1.0']


  0%|          | 4/4727 [00:00<00:06, 757.50it/s]

text: Farmers
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
0 6
text: U.S.
ent_gold: United States
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:118-121
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
118 121
text: farmers
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:123-129
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
123 129
text: They
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:311-314
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
311 314
text: Bloomberg
ent_gold: Bloomberg
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:397-405
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
397 405
